In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('train.csv')

#To extract titles from Name data
def get_titles(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return "unknown"

#To classify other unusual titles to usual ones..
def short_title(x):
    title = x['Title']
    if title in ['Capt','Col','Major']:
        return 'Officer'
    elif title in  ['Sir','Don','Dona','Jonkheer','Lady','the Countess','Dr']:
        return 'Royalty'
    elif title in ['Mlle','Ms']:
        return 'Miss'
    elif title in ['Mme']:
        return 'Mrs'
    else:
        return title

df['Title'] = df['Name'].map(lambda x: get_titles(x))
df['Title'] = df.apply(short_title, axis=1)

#Removing null values and less priority columns
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna("S", inplace=True)
df.drop("Cabin", axis=1, inplace=True)
df.drop("Name", axis=1, inplace=True)
df.drop("Ticket", axis=1, inplace=True)

#Converting String data into numbers --> for ML to read comfortably
df.Sex.replace(('male','female'), (0,1), inplace=True)
df.Embarked.replace(('S','C','Q'), (0,1,2), inplace=True)
df.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'),(0,1,2,3,4,5,6,7), inplace=True)


x = df.drop(['Survived','PassengerId'], axis=1)
y = df['Survived']
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.1)
randomforest = RandomForestClassifier()
randomforest.fit(x_train,y_train)

pickle.dump(randomforest, open('titanic_model.sav','wb'))


In [6]:
def pred_model(pclass,sex,age,sibsp,parch,fare,embarked,title):
    import pickle
    x = [[pclass,sex,age,sibsp,parch,fare,embarked,title]]
    randomforest = pickle.load(open('titanic_model.sav','rb'))
    predictions = randomforest.predict(x)
    print(predictions)

In [7]:
pred_model(1,1,11,1,1,19,1,1)

[1]
